In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
import numpy as np
import datetime
import time
import os

In [3]:
DataSource_df = pd.read_excel('../DataSource/竞品数据源.xlsx')
links_df = pd.DataFrame()
links_df['Original Links'] = DataSource_df['goods_id'].apply(lambda x: 'https://item.taobao.com/item.htm?id='+str(x))
links_df.drop_duplicates(inplace =True)

In [4]:
desired_capabilities = DesiredCapabilities.CHROME  # 修改页面加载策略
desired_capabilities["pageLoadStrategy"] = "none"  # 注释这两行会导致最后输出结果的延迟，即等待页面加载完成再输出

In [5]:
# path = 'C:\\Users\\EDZ\\Spider'# 注意这个路径需要时可执行路径（chmod 777 dir or 755 dir）
# executable_path=path
options = webdriver.ChromeOptions()
prefs = {
        'profile.default_content_setting_values': {
            'images': 2,    }
    }
options.add_experimental_option('prefs', prefs)
# chrome_options = Options()
# chrome_options.add_argument('--headless')#无头模式

browser = webdriver.Chrome(options = options)
# 打开淘宝登录先登录
browser.get("https://login.taobao.com/member/login.jhtml?spm=a21bo.2017.754894437.1.5af911d9q07jcU&f=top&redirectURL=https%3A%2F%2Fwww.taobao.com%2F")
# 定义js代码
script = "Object.defineProperty(navigator,'webdriver',{get: ()=> false,});"
# 执行js代码
browser.execute_script(script)#这样登录的时候就不会出现滑块
# browser.implicitly_wait(2)

In [7]:
total_count = 0
fist_to_csv = 1
item_list = []
date = datetime.date.today()

In [8]:
for link in links_df['Original Links']:
    browser.get(link)
    time.sleep(4)
    currentPageUrl = browser.current_url
    if currentPageUrl.find('tmall')>0:
        platform = 'tm'
#         tm_df = tm_df.append([{'链接':currentPageUrl,'web':'tm'}],ignore_index=True)
#天猫解析部分
        try:
            title = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[1]/h1').text#标题
        except NoSuchElementException:
            title='商品不存在'
#         title_list.append(title)

        try:
            sold_out_flag = browser.find_element_by_xpath('//*[@id="J_Sold-out-recommend"]/div[1]/div[1]/strong').text#是否下架
            sold_out_flag = 1
        except NoSuchElementException:    
            sold_out_flag = 0

        try:
            store_name = browser.find_element_by_xpath('//*[@id="shopExtra"]/div[1]/a').text#店铺名称
        except NoSuchElementException:
            store_name = '天猫超市'


        try:
            store_link =  browser.find_element_by_xpath('//*[@id="shopExtra"]/div[1]/a').get_attribute("href")#店铺链接
        except NoSuchElementException:
            store_link = 'https://chaoshi.tmall.com/'



        try:
            underline_price = browser.find_element_by_xpath('//*[@id="J_StrPriceModBox"]/dd/span[1]').text#划线价
        except NoSuchElementException:
            underline_price = np.nan

        try:
            prmote_price = browser.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text#促销价
        except NoSuchElementException:
            prmote_price = np.nan


        try:
            activity_price = browser.find_element_by_xpath('//*[@id="J_ActivityPrice"]').text#活动价
        except NoSuchElementException:
            activity_price =np.nan

        try:
            act_detail = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[2]/div/ul').get_attribute('innerText')#活动详细信息
            act_detail = act_detail.replace(' ', '')
            act_detail = act_detail.strip()
        except NoSuchElementException:
            act_detail = np.nan
        except WebDriverException:
            act_detail = np.nan

        try:
            browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dt').click()
            time.sleep(0.3)
            activity_info = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dd').text#活动信息
        except NoSuchElementException:
            activity_info = np.nan

        try:
            coupon_info = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/ul/li[3]/div/a').text#优惠券信息
        except NoSuchElementException:
            coupon_info =np.nan

        try:
            sales = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/ul/li[1]/div/span[2]').text#销量
        except NoSuchElementException:
            sales =np.nan

        try:
            comment_num = browser.find_element_by_xpath('//*[@id="J_ItemRates"]/div/span[2]').text#评论数
        except NoSuchElementException: 
            comment_num=np.nan

#######################################################################################################################################  
#淘宝解析部分
    else:
        platform = 'tb'
        try:
            title = browser.find_element_by_xpath('//*[@id="J_Title"]/h3').text#标题
        except NoSuchElementException:
            title='商品不存在'
#         title_list.append(title)



        try:#尝试全球购解析
            store_name = browser.find_element_by_xpath('//*[@id="header-content"]/div[2]/div[1]/div[1]/a').text#店铺名称
        except NoSuchElementException:
            try:#尝试一般淘宝解析
                store_name = browser.find_element_by_xpath('//*[@id="J_ShopInfo"]/div/div[1]/div[1]').text#店铺名称
            except NoSuchElementException:
                store_name = np.nan
#         store_name_list.append(store_name)

        try:
            sold_out_flag = browser.find_element_by_xpath('//*[@id="J_Sold-out-recommend"]/div[1]/div[1]/strong').text#是否下架
            sold_out_flag = 1
        except NoSuchElementException:    
            sold_out_flag = 0
#         is_sold_out.append(sold_out_flag)

        try:#尝试全球购店铺链接
            store_link =  browser.find_element_by_xpath('//*[@id="header-content"]/div[2]/div[1]/div[1]/a').get_attribute("href")#店铺链接
        except NoSuchElementException:
            try:#尝试一般淘宝店铺链接
                store_link =  browser.find_element_by_xpath('//*[@id="J_ShopInfo"]/div/div[1]/div[1]/dl/dd/strong/a').get_attribute("href")#店铺链接
            except NoSuchElementException:
                store_link = np.nan
#         store_link_list.append(store_link)

        try:
            underline_price = browser.find_element_by_xpath('//*[@id="J_StrPrice"]/em[2]').text#划线价
        except NoSuchElementException:
            underline_price = np.nan
#         underline_price_list.append(underline_price)

        try:
            prmote_price = browser.find_element_by_xpath('//*[@id="J_PromoPriceNum"]').text#促销价
        except NoSuchElementException:
            prmote_price = np.nan
#         prmote_price_list.append(prmote_price)

        try:
            activity_price = browser.find_element_by_xpath('//*[@id="J_ActivityPrice"]').text#活动价
        except NoSuchElementException:
            activity_price = np.nan


        try:
            act_detail = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[2]/div/ul').get_attribute('innerText')#活动详细信息
            act_detail = act_detail.replace(' ', '')
            act_detail = act_detail.strip()
        except NoSuchElementException:
            act_detail = np.nan
        except WebDriverException:
            act_detail = np.nan


        try:
            browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dt').click()
            print('找到点击元素')
            time.sleep(0.3)
            activity_info = browser.find_element_by_xpath('//*[@id="J_PromoType"]').text#活动信息
        except NoSuchElementException:
            activity_info = np.nan

        try:
            coupon_info = browser.find_element_by_xpath('//*[@id="J_OtherDiscount"]/div').text#优惠券信息
        except NoSuchElementException:  
            coupon_info = np.nan
#         coupon_info_list.append(coupon_info)

        try:
            sales = browser.find_element_by_xpath('//*[@id="J_SellCounter"]').text#销量
        except NoSuchElementException:
            sales = np.nan
#         sales_list.append(sales)

        try:
            comment_num = browser.find_element_by_xpath('//*[@id="J_TabBar"]/li[2]/a/em').text#评论数
        except NoSuchElementException:
            comment_num = np.nan
#         comment_num_list.append(comment_num)

    item ={'商品原始链接':link,
      '商品最终链接':currentPageUrl,
    '商品标题':title,
      '店铺名称':store_name,
      '店铺链接':store_link,
      '平台':platform,
      '是否下架':sold_out_flag,
     '划线价':underline_price,
     '活动价':activity_price,
     '促销价':prmote_price,
     '活动信息':activity_info,
     '详细活动信息':act_detail,
     '优惠券信息':coupon_info,
     '月销量':sales,
     '累计评论':comment_num,
     '日期':date}
    item_list.append(item)

    if (total_count+1)%5 == 0:
        if fist_to_csv == 1:
            fist_to_csv = 0
            pd.DataFrame(item_list).to_csv('C:\\Users\\EDZ\\Spider\\Result\\'+str(date)+'.csv',header =True,index =False,encoding = 'gbk')
        else:
            pd.DataFrame(item_list).to_csv('C:\\Users\\EDZ\\Spider\\Result\\'+str(date)+'.csv',mode='a', header=False,index =False,encoding = 'gbk')
        item_list = []

    if  (total_count+1)%10 == 0: 
        print('已经爬取'+str(total_count))   
    if (total_count+1)%80 == 0:
        print('休息一刻钟。。。。')
        time.sleep(400)

    total_count =  total_count+1

已经爬取9
已经爬取19
已经爬取29
已经爬取39
已经爬取49
已经爬取59
已经爬取69
已经爬取79
休息一刻钟。。。。
已经爬取89
已经爬取99
已经爬取109
已经爬取119
已经爬取129
已经爬取139
已经爬取149
已经爬取159
休息一刻钟。。。。
已经爬取169
已经爬取179
已经爬取189
已经爬取199
已经爬取209
已经爬取219
已经爬取229
已经爬取239
休息一刻钟。。。。
已经爬取249
已经爬取259
已经爬取269
已经爬取279
已经爬取289
已经爬取299
已经爬取309
已经爬取319
休息一刻钟。。。。
已经爬取329


In [9]:
d = pd.read_csv('C:\\Users\\EDZ\\Spider\\Result\\'+str(date)+'.csv',encoding = 'gbk')
d['划线价'] = d['划线价'].astype('str')
d['划线价'] = d['划线价'].apply(lambda x: x.split('-')[0])
d['促销价'] = d['促销价'].astype('str')
d['促销价'] = d['促销价'].apply(lambda x: x.split('-')[0])

In [10]:
d.to_excel('C:\\Users\\EDZ\\Spider\\Result\\'+str(date)+'.xlsx',header =True,index =False,encoding = 'gbk')

In [ ]:
import datetime
import threading
 
def func():
  print("haha")
  #如果需要循环调用，就要添加以下方法
  timer = threading.Timer(86400, func)
  timer.start()
 
# 获取现在时间
now_time = datetime.datetime.now()
# 获取明天时间
next_time = now_time + datetime.timedelta(days=+1)
next_year = next_time.date().year
next_month = next_time.date().month
next_day = next_time.date().day
# 获取明天3点时间
next_time = datetime.datetime.strptime(str(next_year)+"-"+str(next_month)+"-"+str(next_day)+" 03:00:00", "%Y-%m-%d %H:%M:%S")
# # 获取昨天时间
# last_time = now_time + datetime.timedelta(days=-1)
 
# 获取距离明天3点时间，单位为秒
timer_start_time = (next_time - now_time).total_seconds()
print(timer_start_time)
# 54186.75975
 
#定时器,参数为(多少时间后执行，单位为秒，执行的方法)
timer = threading.Timer(timer_start_time, func)
timer.start()